In [1]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.3 MB/s eta 0:00:00


In [2]:
# app.py
import streamlit as st
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

def predict_high_low(ticker, start_date, prediction_date):
    # Load historical data for the given ticker
    data = yf.download(ticker, start=start_date, end=prediction_date)
    data['Return'] = data['Close'].pct_change()

    # Calculate moving averages
    data['SMA20'] = data['Close'].rolling(window=20).mean()
    data['SMA50'] = data['Close'].rolling(window=50).mean()

    # Calculate Bollinger Bands
    data['StdDev'] = data['Close'].rolling(window=20).std()
    data['UpperBand'] = data['SMA20'] + (data['StdDev'] * 2)
    data['LowerBand'] = data['SMA20'] - (data['StdDev'] * 2)

    # Calculate Average True Range (ATR)
    data['High-Low'] = data['High'] - data['Low']
    data['High-PrevClose'] = abs(data['High'] - data['Close'].shift())
    data['Low-PrevClose'] = abs(data['Low'] - data['Close'].shift())
    data['TrueRange'] = data[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)
    data['ATR'] = data['TrueRange'].rolling(window=14).mean()

    # Get the data for the prediction date
    latest_close = data['Close'].iloc[-1]
    latest_atr = data['ATR'].iloc[-1]

    # Calculate the predicted high and low for the next day
    predicted_high = latest_close + latest_atr
    predicted_low = latest_close - latest_atr

    return predicted_high, predicted_low, data

st.title('QQQ High and Low Prediction')

ticker = st.text_input('Enter the ticker symbol', 'QQQ')
start_date = st.text_input('Enter the start date for historical data (YYYY-MM-DD)', '2023-01-01')
prediction_date = st.text_input('Enter the date for prediction (YYYY-MM-DD)', '2024-07-01')

if st.button('Predict'):
    predicted_high, predicted_low, data = predict_high_low(ticker, start_date, prediction_date)
    st.write(f"Predicted high for the day after {prediction_date}: {predicted_high}")
    st.write(f"Predicted low for the day after {prediction_date}: {predicted_low}")

    # Plot historical data and predicted range
    plt.figure(figsize=(12, 6))
    plt.plot(data['Close'], label='Close Price')
    plt.plot(data['SMA20'], label='20-Day SMA')
    plt.plot(data['UpperBand'], label='Upper Bollinger Band', linestyle='--')
    plt.plot(data['LowerBand'], label='Lower Bollinger Band', linestyle='--')
    plt.axhline(y=predicted_high, color='r', linestyle='-', label='Predicted High')
    plt.axhline(y=predicted_low, color='g', linestyle='-', label='Predicted Low')
    plt.title(f'{ticker} Historical Prices with Predicted High and Low for {prediction_date}')
    plt.legend()
    st.pyplot(plt)


2024-07-03 00:56:28.705 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-07-03 00:56:28.708 Session state does not function when running a script without `streamlit run`


In [3]:
!streamlit run app.py


Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py
